In [1]:
import pandas as pd
from chanlun import cl
from chanlun import kcharts

### 避免在 Notebook 中不显示，手动加载下依赖的js
###　https://pyecharts.org/#/zh-cn/notebook
from pyecharts.charts import Kline, Line, Bar, Grid, Scatter
from pyecharts.globals import CurrentConfig, NotebookType

CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB  # 根据使用的环境设置 notebook 类型， NotebookType.JUPYTER_LAB 或 NotebookType.JUPYTER_NOTEBOOK
Kline().load_javascript()
Line().load_javascript()
Bar().load_javascript()
Grid().load_javascript()
Scatter().load_javascript()

## 缠论配置项
参考项目的 src/chanlun/cl_interface.py 的 Config 对象

---

### 缠论的默认配置项


    # 分型默认配置
    'fx_qj': Config.FX_QJ_K.value,
    'fx_bh': Config.FX_BH_DINGDI.value,
    # 笔默认配置
    'bi_type': Config.BI_TYPE_NEW.value,
    'bi_bzh': Config.BI_BZH_YES.value,
    'bi_qj': Config.BI_QJ_CK.value,
    'bi_fx_cgd': Config.BI_FX_CHD_NO.value,
    # 线段默认配置
    'xd_bzh': Config.XD_BZH_YES.value,
    'xd_qj': Config.XD_QJ_DD.value,
    # 走势类型默认配置
    'zslx_bzh': Config.ZSLX_BZH_YES.value,
    'zslx_qj': Config.ZSLX_QJ_DD.value,
    # 中枢默认配置
    'zs_bi_type': Config.ZS_TYPE_DN.value,  # 笔中枢类型
    'zs_xd_type': Config.ZS_TYPE_DN.value,  # 走势中枢类型
    'zs_qj': Config.ZS_QJ_DD.value,
    'zs_wzgx': Config.ZS_WZGX_ZGD.value,
    # 指标配置项
    'idx_macd_fast': 12,
    'idx_macd_slow': 26,
    'idx_macd_signal': 9,
    'idx_boll_period': 20,
    'idx_ma_period': 5,

In [2]:
# 获取 行情K线数据
code = 'SH.688122'
frequency = '30m'
# 缠论配置
cl_config = {'bi_type': 'new', 'xd_bzh': 'xd_bzh_yes', 'zs_bi_type': 'zs_type_dn', 'idx_macd_fast': 12}
# 缠论画图配置
chart_config = {'show_bi_zs': True, 'show_ma': False}
klines = pd.read_csv('./data/688122.csv')
klines.tail()


,code,date,open,close,high,low,volume
2395,SH.688122,2022-04-08 11:30:00,82.00,81.50,82.59,81.27,178700.0
2396,SH.688122,2022-04-08 13:30:00,81.51,81.51,82.08,81.16,180800.0
2397,SH.688122,2022-04-08 14:00:00,81.52,81.40,81.78,81.12,142800.0
2398,SH.688122,2022-04-08 14:30:00,81.31,81.72,81.80,80.18,305800.0
2399,SH.688122,2022-04-08 15:00:00,81.80,82.85,83.03,81.74,1229100.0


In [3]:
# 依据 K 线数据，计算缠论数据
cl_data = cl.CL(code, frequency, cl_config).process_klines(klines)
chart = kcharts.render_charts('%s - %s' % (code, frequency), cl_data, config=chart_config)

In [ ]:
# 展示图标
chart

In [5]:
# 获取最后一笔
bi = cl_data.bis[-1]
print('Last Bi : ', bi)
# 获取最后一个中枢
zs = cl_data.bi_zss[-1]
print('Last Zs : ', zs)

Last Bi :  index: 552 type: down FX: (2022-04-08 11:00:00 - 2022-04-08 14:30:00) high: 82.87 low: 80.18 done: True
Last Zs :  index: 110 zs_type: bi level: 0 FX: (2022-03-29 10:30:00-2022-03-31 10:30:00) type: up zg: 87.37 zd: 85.8 gg: 88.5 dd: 84.66 done: True


In [6]:
# 判断笔是否背驰，是否有买卖点 （只在 Pro 版本支持）
bi = cl_data.bis[-11]
print('BI MMDS : ', bi.line_mmds())
print('BI BC : ', [_bc.type for _bc in bi.bcs if _bc.bc])
print('BI 笔背驰：', bi.bc_exists(['bi']))
print('BI 盘整背驰：', bi.bc_exists(['pz']))
print('BI 趋势背驰：', bi.bc_exists(['qs']))

BI MMDS :  []
BI BC :  []
BI 笔背驰： False
BI 盘整背驰： False
BI 趋势背驰： False


In [7]:
# 买卖点信息 (只在 Pro 版本支持)
# mmd = bi.mmds[0]
# print(mmd)
# print('买卖点对应的中枢：', mmd.zs)

## Html 页面展示

### 服务端代码
    return HttpResponse(kcharts.render_charts('标题', cl_datas))

### 页面 Ajax 接收并展示
    var chart = echarts.init(document.getElementById('chart_id'), 'drak', {renderer: 'canvas'});
    $.ajax({
        type: "GET",
        url: "/后端请求地址",
        success: function (result) {
            var res = (new Function("return " + result))();
            chart.setOption(res);
        }
    });